In [ ]:
from pyspark.sql import SparkSession 
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [417]:
TOPIC = 'groupB04.spotifytempos'

In [431]:
df = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "node1:9092") \
        .option("subscribePattern", TOPIC) \
        .option("startingOffsets", "earliest")  \
        .load()
spark.stop()

In [430]:
df.printSchema()
df.select("*").show()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



AnalysisException: Queries with streaming sources must be executed with writeStream.start();;
kafka

In [420]:
from pyspark.sql.functions import get_json_object, concat, lit
from pyspark.sql.types import *
from pyspark.sql import functions as f

#values = converted['value'].apply(lambda x: str(x)[2:-2].split(','))
split_col = f.split(df['value'], ',')
decade = concat(split_col.getItem(1)[0:3], lit(0))


df = df \
         .withColumn('UUID', split_col.getItem(0)) \
         .withColumn('year', split_col.getItem(1)) \
         .withColumn('tempo', split_col.getItem(2)) \
         .withColumn('decade', decade) \
         .sort(f.col("offset"),f.col("partition").asc())

converted = df \
    .selectExpr("UUID", "year", "tempo", "partition", "offset", "decade")\
    .select([\
            df["UUID"].cast(StringType()), \
            df["year"].cast(IntegerType()), \
            df["tempo"].cast(DoubleType()), \
            df["partition"].cast(IntegerType()), \
            df["offset"].cast(IntegerType()), \
            df["decade"].cast(IntegerType())\
            ])

In [421]:
converted.toPandas()

,UUID,year,tempo,partition,offset,decade
0,4EafdWyeDb7Xlvhs7Xgca8,1936,100.550,2,0,1930
1,0NQnCpLQYQc4o7SLhgSkBB,1994,99.890,5,0,1990
2,4h8gqHQ6pdgXWqElxBSGeo,1935,126.048,6,0,1930
3,48U33WgDS5EEUKEhMT9Zwt,1948,85.126,10,0,1940
4,4grD2NkyBWSjo15u0muqA2,1959,82.888,11,0,1950
...,...,...,...,...,...,...
106,6pK1qZ0N8FJeEhmij0HDYP,1971,114.588,36,3,1970
107,29UUG0DJA765U77TFJKvuc,1962,63.971,77,3,1960
108,1mFHi6mwIPAa82RSduebmj,1964,147.494,11,4,1960
109,6bopTr1MVZmpStZjzkbNkE,1961,113.320,14,4,1960


In [422]:
groupdf = converted \
    .select("partition", "year", "decade", "tempo", "offset") \
    .groupby("year") \
    .agg( \
         f.avg("tempo").alias("avg"), \
         f.count("partition").alias("count"), \
         f.max("offset").alias("offsetMax"), 
         f.max("decade").alias("decade"))\
    .sort("year")


In [423]:
groupdf.toPandas().tail()

,year,avg,count,offsetMax,decade
76,2016,82.027,1,0,2010
77,2017,74.369,1,2,2010
78,2018,116.021,1,0,2010
79,2019,90.013,1,0,2010
80,2020,90.048,1,0,2020


In [424]:
groupdf_second = groupdf \
    .select("avg", "decade") \
    .groupby("decade") \
    .agg( \
         f.avg("avg").alias("average tempo")) \
    .sort("decade")

In [425]:
groupdf_second.toPandas()

,decade,average tempo
0,1920,115.970000
1,1930,115.782000
2,1940,106.786643
3,1950,97.003400
4,1960,113.202450
5,1970,115.582850
6,1980,117.339000
7,1990,123.863667
8,2000,134.879100
9,2010,114.290700


In [428]:
%matplotlib inline
from pyspark.sql import SparkSession
from IPython import display
import matplotlib.pyplot as plt

plt

AttributeError: 'DataFrame' object has no attribute 'barplot'